# Setup to allow tests in notebooks

Notice this setup uses:

- Azure Open AI
- Azure Search Service

The idea behind these Tests is to modulate the role of system to the required response

In [1]:
import os
import openai # this will require pip install openai
import tiktoken # this will require pip install tiktoken
import requests

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [2]:
# On Your Data Settings
DATASOURCE_TYPE = os.environ.get("DATASOURCE_TYPE", "AzureCognitiveSearch")
SEARCH_TOP_K = os.environ.get("SEARCH_TOP_K", 5)
SEARCH_STRICTNESS = os.environ.get("SEARCH_STRICTNESS", 3)
SEARCH_ENABLE_IN_DOMAIN = os.environ.get("SEARCH_ENABLE_IN_DOMAIN", "true")

# ACS Integration Settings
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_SEARCH_KEY = os.environ.get("AZURE_SEARCH_KEY")
AZURE_SEARCH_USE_SEMANTIC_SEARCH = os.environ.get("AZURE_SEARCH_USE_SEMANTIC_SEARCH", "false")
AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG = os.environ.get("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG", "default")
AZURE_SEARCH_TOP_K = os.environ.get("AZURE_SEARCH_TOP_K", SEARCH_TOP_K)
AZURE_SEARCH_ENABLE_IN_DOMAIN = os.environ.get("AZURE_SEARCH_ENABLE_IN_DOMAIN", SEARCH_ENABLE_IN_DOMAIN)
AZURE_SEARCH_CONTENT_COLUMNS = os.environ.get("AZURE_SEARCH_CONTENT_COLUMNS")
AZURE_SEARCH_FILENAME_COLUMN = os.environ.get("AZURE_SEARCH_FILENAME_COLUMN")
AZURE_SEARCH_TITLE_COLUMN = os.environ.get("AZURE_SEARCH_TITLE_COLUMN")
AZURE_SEARCH_URL_COLUMN = os.environ.get("AZURE_SEARCH_URL_COLUMN")
AZURE_SEARCH_VECTOR_COLUMNS = os.environ.get("AZURE_SEARCH_VECTOR_COLUMNS")
AZURE_SEARCH_QUERY_TYPE = os.environ.get("AZURE_SEARCH_QUERY_TYPE")
AZURE_SEARCH_PERMITTED_GROUPS_COLUMN = os.environ.get("AZURE_SEARCH_PERMITTED_GROUPS_COLUMN")
AZURE_SEARCH_STRICTNESS = os.environ.get("AZURE_SEARCH_STRICTNESS", SEARCH_STRICTNESS)

# AOAI
AZURE_OPENAI_RESOURCE = os.environ.get("AZURE_OPENAI_RESOURCE")
AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_MODEL = os.environ.get("AZURE_OPENAI_MODEL")
AZURE_OPENAI_SYSTEM_MESSAGE = os.environ.get("AZURE_OPENAI_SYSTEM_MESSAGE", "You are an AI assistant that helps people find information.")

openai.api_key  = AZURE_OPENAI_KEY
openai.api_type = "azure"
openai.api_version = "2023-08-01-preview"
openai.api_base = f"https://{AZURE_OPENAI_RESOURCE}.openai.azure.com/"


In [3]:
def print_required_variables(debug=True):
    if debug:
        print(f"AZURE_SEARCH_SERVICE: {AZURE_SEARCH_SERVICE}")
        print(f"AZURE_SEARCH_INDEX: {AZURE_SEARCH_INDEX}")
        print(f"AZURE_SEARCH_KEY: {AZURE_SEARCH_KEY}")
        print(f"AZURE_SEARCH_USE_SEMANTIC_SEARCH: {AZURE_SEARCH_USE_SEMANTIC_SEARCH}")
        print(f"AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG: {AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG}")
        print(f"AZURE_SEARCH_TOP_K: {AZURE_SEARCH_TOP_K}")
        print(f"AZURE_SEARCH_ENABLE_IN_DOMAIN: {AZURE_SEARCH_ENABLE_IN_DOMAIN}")
        print(f"AZURE_SEARCH_CONTENT_COLUMNS: {AZURE_SEARCH_CONTENT_COLUMNS}")
        print(f"AZURE_SEARCH_FILENAME_COLUMN: {AZURE_SEARCH_FILENAME_COLUMN}")
        print(f"AZURE_SEARCH_TITLE_COLUMN: {AZURE_SEARCH_TITLE_COLUMN}")
        print(f"AZURE_SEARCH_URL_COLUMN: {AZURE_SEARCH_URL_COLUMN}")
        print(f"AZURE_SEARCH_VECTOR_COLUMNS: {AZURE_SEARCH_VECTOR_COLUMNS}")
        print(f"AZURE_SEARCH_QUERY_TYPE: {AZURE_SEARCH_QUERY_TYPE}")
        print(f"AZURE_SEARCH_PERMITTED_GROUPS_COLUMN: {AZURE_SEARCH_PERMITTED_GROUPS_COLUMN}")
        print(f"AZURE_SEARCH_STRICTNESS: {AZURE_SEARCH_STRICTNESS}")
        print(f"AZURE_OPENAI_RESOURCE: {AZURE_OPENAI_RESOURCE}")
        print(f"AZURE_OPENAI_KEY: {AZURE_OPENAI_KEY}")
        print(f"AZURE_OPENAI_MODEL: {AZURE_OPENAI_MODEL}")
        print(f"AZURE_OPENAI_SYSTEM_MESSAGE: {AZURE_OPENAI_SYSTEM_MESSAGE}")

In [5]:
print_required_variables(True)

AZURE_SEARCH_SERVICE: gptkb-moww6huavlklg
AZURE_SEARCH_INDEX: people-culture-faq
AZURE_SEARCH_KEY: QqNRt5LDccm2evQgIViI3EYGJTmePwDPFkyqu1kQz8AzSeDK1oYt
AZURE_SEARCH_USE_SEMANTIC_SEARCH: True
AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG: default
AZURE_SEARCH_TOP_K: 5
AZURE_SEARCH_ENABLE_IN_DOMAIN: True
AZURE_SEARCH_CONTENT_COLUMNS: content
AZURE_SEARCH_FILENAME_COLUMN: filepath
AZURE_SEARCH_TITLE_COLUMN: title
AZURE_SEARCH_URL_COLUMN: url
AZURE_SEARCH_VECTOR_COLUMNS: None
AZURE_SEARCH_QUERY_TYPE: semantic
AZURE_SEARCH_PERMITTED_GROUPS_COLUMN: None
AZURE_SEARCH_STRICTNESS: 3
AZURE_OPENAI_RESOURCE: cog-moww6huavlklg
AZURE_OPENAI_KEY: 67062ab6dcde4f9ab8243f04601e6e88
AZURE_OPENAI_MODEL: chat
AZURE_OPENAI_SYSTEM_MESSAGE: Eres un asistente de recursos humanos. Tu respuesta debe ser en español y debe siempre empezar por estimado colaborador


In [10]:
def get_completion_from_messages(messages,
                                 role_value,
                                 debug=False,
                                 show_completion=False) :
    if debug:
        print_required_variables(debug)
        print(f"messages: {messages}")
    
    def setup_byod(deployment_id: str) -> None:
        """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
        :param deployment_id: The deployment ID for the model to use with your own data.
        To remove this configuration, simply set openai.requestssession to None.
        """

        class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

            def send(self, request, **kwargs):
                request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
                return super().send(request, **kwargs)

        session = requests.Session()

        # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
        session.mount(
            prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
            adapter=BringYourOwnDataAdapter()
        )
        
        openai.requestssession = session

    setup_byod(AZURE_OPENAI_MODEL)
    search_endpoint = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
    
    completion = openai.ChatCompletion.create(
        messages=messages,
        deployment_id=AZURE_OPENAI_MODEL,
        dataSources=[  # camelCase is intentional, as this is the format the API expects
            {
                "type": DATASOURCE_TYPE,
                "parameters": {
                    "endpoint": search_endpoint,
                    "key": AZURE_SEARCH_KEY,
                    "indexName": AZURE_SEARCH_INDEX,
                    "fieldsMapping": {
                        "contentFields": AZURE_SEARCH_CONTENT_COLUMNS.split("|") if AZURE_SEARCH_CONTENT_COLUMNS else [],
                        "titleField": AZURE_SEARCH_TITLE_COLUMN if AZURE_SEARCH_TITLE_COLUMN else None,
                        "urlField": AZURE_SEARCH_URL_COLUMN if AZURE_SEARCH_URL_COLUMN else None,
                        "filepathField": AZURE_SEARCH_FILENAME_COLUMN if AZURE_SEARCH_FILENAME_COLUMN else None,
                        "vectorFields": AZURE_SEARCH_VECTOR_COLUMNS.split("|") if AZURE_SEARCH_VECTOR_COLUMNS else []
                    },
                    "inScope": True if AZURE_SEARCH_ENABLE_IN_DOMAIN.lower() == "true" else False,
                    "topNDocuments": AZURE_SEARCH_TOP_K,
                    "queryType": AZURE_SEARCH_QUERY_TYPE,
                    "semanticConfiguration": AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG if AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG else "",
                    "roleInformation": role_value,
                    "strictness": int(AZURE_SEARCH_STRICTNESS)
                }
            }
        ]
    )

    if show_completion:
        print(completion)

    return completion.choices[0].message["content"]


In [7]:
user_message = f"""\
Lista los beneficios de Baufest"""

messages =  [  
{'role':'user', 
 'content': f"{user_message}"},  
] 

response = get_completion_from_messages(messages,show_completion=False)
print(response)

Estimado colaborador, Baufest ofrece varios beneficios a sus empleados en diferentes áreas. A continuación, se detallan algunos de ellos:

- BAU Lunch (LATAM): Este beneficio ofrece a los empleados de Argentina, Perú y Chile la posibilidad de recibir acreditaciones para almuerzos a través de la aplicación PedidosYa o Tickets Restaurant emitidos por Edenred. La acreditación es semanal en Argentina y Perú, y mensual en Uruguay. [doc1]
- BAU Health (LATAM): Este beneficio permite a los empleados de Baufest en Latinoamérica solicitar un reembolso por actividades físicas realizadas en el trimestre correspondiente, siempre y cuando cumplan con los requisitos establecidos. [doc3]
- BAU Net (LATAM): Este beneficio ofrece a los empleados de Baufest en Latinoamérica la posibilidad de solicitar un reembolso por el servicio de internet en su hogar, siempre y cuando cumplan con los requisitos establecidos. [doc3]
- BAU Dollar: Este beneficio ofrece a los empleados de Baufest la posibilidad de recib

### Escenario datos usuario, no hay informacion en documento

Necesitamos las siguientes variables

user_question
system_message
delimiter
user_prompt
user_context

In [15]:
delimiter = "'''"
system_message = f"""
Eres un asistente de People and Culture y tu trabajo es buscar información \
la pregunta está delimitada por tres comillas simples. 

Si dentro de la pregunta no viene el nombre, debes de pedirle su nombre
"""

In [16]:
user_question = f"""\
Lista los beneficios de Baufest"""
print(f"{delimiter}{user_question}{delimiter}")
print(f"{system_message}")


'''Lista los beneficios de Baufest'''

Eres un asistente de People and Culture y tu trabajo es buscar información la pregunta está delimitada por tres comillas simples. 

Si dentro de la pregunta no viene el nombre, debes de pedirle su nombre



In [17]:


messages =  [  
{'role':'system', 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_question}{delimiter}"},  
] 

response = get_completion_from_messages(messages,system_message)
print(response)

Los beneficios de Baufest pueden variar según la región, pero algunos de ellos incluyen el programa de almuerzo, el cual es mensual y se acredita incluso en días feriados . También se ofrece un beneficio de vivienda, el cual requiere una antigüedad de dos años en la empresa y la presentación de un contrato de alquiler o escritura a nombre del empleado o un documento legal que respalde la convivencia [doc1]. Además, se ofrece un beneficio de tráfico, el cual permite cambiar el rango de horario una vez que ya se ha optado por uno [doc1]. En cuanto a los beneficios en dólares, se pueden abrir cuentas sueldo en diferentes bancos y plataformas, como HSBC, Facebank, Wise y Grabr FI, y se recomienda consultar periódicamente los costos, ya que pueden sufrir modificaciones [doc2]. No se pueden hacer transferencias a billeteras virtuales, cuentas comitentes o plataformas no bancarias, y es necesario hacer la transferencia a una cuenta bancaria a nombre del empleado [doc2]. Por último, no se pued